# Title: 🚗 **Saudi Arabia Used-Car Price Optimization for Syarah.com**

---

## 🎯 **SCQA: Business Framing**

**Situation:**
Sellers on syarah.com, an online used-car marketplace in Saudi Arabia, regularly list vehicles for sale. Setting the right asking price is crucial for achieving timely sales and maintaining marketplace efficiency.

**Complication:**
Many sellers inaccurately price their vehicles due to limited market insights. Overpriced cars lead to slow sales and reduced marketplace liquidity, while underpriced cars result in lost profits for sellers and lower revenues for the platform.

**Question:**
How can syarah.com help sellers reliably set optimal asking prices to minimize pricing errors, accelerate sales, and improve both seller profitability and overall marketplace performance?

**Answer:**
Develop and deploy a robust machine-learning model that predicts accurate market-aligned prices for used cars, significantly reducing pricing errors. This will increase marketplace liquidity, accelerate sales, and maximize profit for both sellers and syarah.com.

# Import Libraries & Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
DATASET_PATH = '../data/data_saudi_used_cars.csv'
CLEANED_DATA_PATH = '../data/cleaned_data_saudi_used_cars.csv'

In [3]:
df = pd.read_csv(DATASET_PATH)
print("Initial shape:", df.shape)
display(df.head())

Initial shape: (5624, 11)


,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,Corolla,Abha,Toyota,Manual,Saudi,Standard,2013,1.4,421000,True,0
1,Yukon,Riyadh,GMC,Automatic,Saudi,Full,2014,8.0,80000,False,120000
2,Range Rover,Riyadh,Land Rover,Automatic,Gulf Arabic,Full,2015,5.0,140000,False,260000
3,Optima,Hafar Al-Batin,Kia,Automatic,Saudi,Semi Full,2015,2.4,220000,False,42000
4,FJ,Riyadh,Toyota,Automatic,Saudi,Full,2020,4.0,49000,True,0


# Data Inspection

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5624 entries, 0 to 5623
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Type         5624 non-null   object 
 1   Region       5624 non-null   object 
 2   Make         5624 non-null   object 
 3   Gear_Type    5624 non-null   object 
 4   Origin       5624 non-null   object 
 5   Options      5624 non-null   object 
 6   Year         5624 non-null   int64  
 7   Engine_Size  5624 non-null   float64
 8   Mileage      5624 non-null   int64  
 9   Negotiable   5624 non-null   bool   
 10  Price        5624 non-null   int64  
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 445.0+ KB


In [5]:
print("Missing values per column:\n", df.isna().sum())
print("\nDuplicate rows:", df.duplicated().sum())

Missing values per column:
 Type           0
Region         0
Make           0
Gear_Type      0
Origin         0
Options        0
Year           0
Engine_Size    0
Mileage        0
Negotiable     0
Price          0
dtype: int64

Duplicate rows: 4


In [6]:
display(df[df.duplicated(keep=False)])

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
236,Sonata,Jeddah,Hyundai,Automatic,Saudi,Standard,2020,2.4,100,False,72910
540,Rio,Riyadh,Kia,Automatic,Saudi,Standard,2016,1.4,270000,False,25000
953,Rio,Riyadh,Kia,Automatic,Saudi,Standard,2016,1.4,270000,False,25000
2012,Tucson,Dammam,Hyundai,Automatic,Saudi,Standard,2016,2.0,155000,False,45000
3631,Camry,Riyadh,Toyota,Automatic,Saudi,Full,2013,2.5,185000,True,0
4646,Sonata,Jeddah,Hyundai,Automatic,Saudi,Standard,2020,2.4,100,False,72910
4933,Tucson,Dammam,Hyundai,Automatic,Saudi,Standard,2016,2.0,155000,False,45000
5267,Camry,Riyadh,Toyota,Automatic,Saudi,Full,2013,2.5,185000,True,0


There are 4 exact duplicates found in the dataset. After inspection, it is safe to remove these duplicates since these duplicates do not contain any meaningful data/context for our predictive model.

In [7]:
df = df.drop_duplicates()
print("Shape after dropping duplicates:", df.shape)

Shape after dropping duplicates: (5620, 11)


# Handling Obvious Dataset Issue

## Excluding Invalid Prices (Price = 0 or Negotiable Prices)

In this dataset, car listings with `"Negotiable" = True` have a `Price` value of **0**, which does not reflect the actual market price. Including these rows would:

- **Distort distribution** of price-related statistics (mean, median, outliers)
- **Skew visualizations** like boxplots or scatter plots
- **Confuse model training**, since a price of 0 is not a valid learning signal

In [8]:
df[df['Negotiable'] == True].head()

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,Corolla,Abha,Toyota,Manual,Saudi,Standard,2013,1.4,421000,True,0
4,FJ,Riyadh,Toyota,Automatic,Saudi,Full,2020,4.0,49000,True,0
9,LS,Aseer,Lexus,Automatic,Saudi,Full,1998,4.6,300,True,0
14,Hilux,Yanbu,Toyota,Automatic,Saudi,Full,2014,2.7,209,True,0
15,Land Cruiser,Aseer,Toyota,Manual,Gulf Arabic,Semi Full,1982,4.6,340000,True,0


In [9]:
# Calculate the number of rows with Negotiable as True
negotiable_true_count = df['Negotiable'].sum()

# Calculate the percentage
negotiable_true_percentage = (negotiable_true_count / len(df)) * 100

print(f"Number of cars with Negotiable = True: {negotiable_true_count}")
print(f"Percentage of cars with Negotiable = True: {negotiable_true_percentage:.2f}%")

# Check how many of these have Price = 0
negotiable_zero_price = df[(df['Negotiable'] == True) & (df['Price'] == 0)]
print(f"\nNumber of negotiable cars with Price = 0: {len(negotiable_zero_price)}")
print(f"Percentage of negotiable cars with Price = 0: {(len(negotiable_zero_price) / negotiable_true_count) * 100:.2f}%")

Number of cars with Negotiable = True: 1795
Percentage of cars with Negotiable = True: 31.94%

Number of negotiable cars with Price = 0: 1795
Percentage of negotiable cars with Price = 0: 100.00%


Let's see how many listings have a price of 0

In [10]:
listings_with_0_price = df[df['Price'] == 0]
print(f"\nNumber of listings with Price = 0: {len(listings_with_0_price)}")


Number of listings with Price = 0: 1796


There are 1796 listings with price = 0, while there are only 1795 listings with Negotiable prices. That means there is a listing with non-negotiable price of 0 (impossible)

In [11]:
listings_with_0_price[listings_with_0_price['Negotiable'] == False]

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
3168,Furniture,Riyadh,Toyota,Automatic,Other,Semi Full,2012,2.7,225000,False,0


This value is considered as invalid as it is not realistic for a person to sell a car for free. Therefore, I will exclude this from the dataset as well.

## Solution:
I exclude listings with `Price = 0` (which includes `Negotiable == True`) from the dataset.

This ensures that I train and evaluate our model only on cars with fixed, known prices, leading to more accurate and reliable insights.

In [ ]:
# Remove price = 0 rows
df_valid_prices = df[df['Price'] > 0]
print("Shape after removing Price = 0 rows:", df_valid_prices.shape)

Shape after removing Price = 0 rows: (3824, 11)


Data before initial cleaning:

In [13]:
df.describe(include='all')

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
count,5620,5620,5620,5620,5620,5620,5620.000000,5620.000000,5.620000e+03,5620,5620.000000
unique,347,27,58,2,4,3,NaN,NaN,NaN,2,NaN
top,Land Cruiser,Riyadh,Toyota,Automatic,Saudi,Full,NaN,NaN,NaN,False,NaN
freq,269,2270,1430,4871,4184,2232,NaN,NaN,NaN,3825,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2014.100356,3.296299,1.509222e+05,NaN,53086.404448
std,NaN,NaN,NaN,NaN,NaN,NaN,5.793003,1.515253,3.829633e+05,NaN,70175.153207
min,NaN,NaN,NaN,NaN,NaN,NaN,1963.000000,1.000000,1.000000e+02,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,2012.000000,2.000000,3.800000e+04,NaN,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2016.000000,3.000000,1.030000e+05,NaN,36500.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,2018.000000,4.500000,1.960000e+05,NaN,73000.000000


Data after initial cleaning:

In [14]:
df_valid_prices.describe(include='all')

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
count,3824,3824,3824,3824,3824,3824,3824.000000,3824.000000,3.824000e+03,3824,3824.000000
unique,320,27,56,2,4,3,NaN,NaN,NaN,1,NaN
top,Camry,Riyadh,Toyota,Automatic,Saudi,Full,NaN,NaN,NaN,False,NaN
freq,149,1615,889,3420,2893,1473,NaN,NaN,NaN,3824,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2014.851203,3.181119,1.426211e+05,NaN,78019.245031
std,NaN,NaN,NaN,NaN,NaN,NaN,5.110916,1.458832,3.783942e+05,NaN,72748.182183
min,NaN,NaN,NaN,NaN,NaN,NaN,1963.000000,1.000000,1.000000e+02,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,2013.000000,2.000000,4.800000e+04,NaN,35000.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2016.000000,2.700000,1.030000e+05,NaN,58000.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,2018.000000,4.000000,1.830000e+05,NaN,94250.000000


After the price filtering, 27 `Type` and 2 `Make` are also indirectly filtered out, meaning those Types and Makes are all negotiable in this dataset and are likely to only have few rows in the original dataset.

In [15]:
# Find Types and Makes present in df but missing in df_filtered
types_removed = set(df['Type'].unique()) - set(df_valid_prices['Type'].unique())
makes_removed = set(df['Make'].unique()) - set(df_valid_prices['Make'].unique())

# Pick one Type and one Make that were filtered out
type_example = next(iter(types_removed))
make_example = next(iter(makes_removed))

# Show all rows from df for the selected Type and Make
print(f"Example of filtered-out Type: {type_example}")
display(df[df['Type'] == type_example])

print(f"\nExample of filtered-out Make: {make_example}")
display(df[df['Make'] == make_example])

Example of filtered-out Type: SLK


,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
2207,SLK,Riyadh,Mercedes,Automatic,Gulf Arabic,Full,2013,1.8,137000,True,0
2909,SLK,Riyadh,Mercedes,Automatic,Gulf Arabic,Full,2013,3.0,100000,True,0



Example of filtered-out Make: Tata


,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
4325,Nexon,Jeddah,Tata,Manual,Saudi,Standard,2016,2.2,256000,True,0


For example, both SLK `Type` and Tata `Make` only have 1 listing in the dataset, and both listings are negotiable.

# Save file for the next workflow

In [18]:
# RESET INDEX
df_valid_prices = df_valid_prices.reset_index(drop=True)
df_valid_prices.head()

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,Yukon,Riyadh,GMC,Automatic,Saudi,Full,2014,8.0,80000,False,120000
1,Range Rover,Riyadh,Land Rover,Automatic,Gulf Arabic,Full,2015,5.0,140000,False,260000
2,Optima,Hafar Al-Batin,Kia,Automatic,Saudi,Semi Full,2015,2.4,220000,False,42000
3,CX3,Abha,Mazda,Automatic,Saudi,Semi Full,2019,2.0,25000,False,58000
4,Cayenne S,Riyadh,Porsche,Automatic,Saudi,Full,2012,4.8,189000,False,85000


In [20]:
# FINAL QUICK CHECKS
print("Final missing values per column:\n", df_valid_prices.isna().sum())
print("\nFinal duplicate rows:", df_valid_prices.duplicated().sum())
print("\nFinal shape after cleaning:", df_valid_prices.shape)

Final missing values per column:
 Type           0
Region         0
Make           0
Gear_Type      0
Origin         0
Options        0
Year           0
Engine_Size    0
Mileage        0
Negotiable     0
Price          0
dtype: int64

Final duplicate rows: 0

Final shape after cleaning: (3824, 11)


In [21]:
# SAVE CLEANED DATA
df_valid_prices.to_csv(CLEANED_DATA_PATH, index=False)
print(f"Cleaned data saved to {CLEANED_DATA_PATH}")

Cleaned data saved to ../data/cleaned_data_saudi_used_cars.csv


## 📊 Initial Data Exploration Summary

### 🧾 Dataset Overview

* **Total rows**: 3,824
* **Columns**: 11 (6 categorical, 1 boolean, 4 numerical)
* **No missing values & removed 4 duplicates**

---

### 🔍 Categorical Features

* **High cardinality**

  * `Type`: 320 unique models
  * `Make`: 56 brands
* **Moderate cardinality**

  * `Region`: 27 areas
* **Dominant categories**

  * **Make**: Toyota (\~23%)
  * **Type**: Camry (\~4%)
  * **Region**: Riyadh (\~42%)
  * **Gear\_Type**: Automatic (\~89%)
  * **Origin**: Saudi (\~76%)
  * **Options**: Full (\~39%)

> ⚠️ Heavy skew: a handful of categories cover most listings; long tail of rare values remains.

---

### 🔢 Numerical Features

| Feature              | Min  | 25 %   | 50 %    | Mean    | 75 %    | Max        |
| -------------------- | ---- | ------ | ------- | ------- | ------- | ---------- |
| **Year**             | 1963 | 2013   | 2016    | 2015    | 2018    | 2021       |
| **Engine\_Size (L)** | 1.0  | 2.0    | 2.7     | 3.18    | 4.0     | 9.0        |
| **Mileage (km)**     | 100  | 48,000 | 103,000 | 142,600 | 183,000 | 20,000,000 |
| **Price (SAR)**      | 1    | 35,000 | 58,000  | 78,019  | 94,250  | 850,000    |

> ⚠️ Mileage and Price have extreme outliers

---

### ✅ Key Takeaways

1. **Filtered out** 1,795 negotiable listings and 1 invalid 0-price listing -> using 3,824 fixed-price entries.
2. **High cardinality** in `Type` & `Make`, with strong region skew -> encoding/aggregation needed.
3. **Long tails & outliers** (Mileage, Price) require attention before modeling.
4. Clean and ready for detailed EDA & preprocessing.